In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib.reproduction import major_oxides
from sklearn.metrics import mean_squared_error
from lib import full_flow_dataloader
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout, Dense 

import tensorflow as tf

from lib.norms import Norm1Scaler, Norm3Scaler

In [ ]:
train_processed, test_processed = full_flow_dataloader.load_full_flow_data(load_cache_if_exits=False, average_shots=True)

In [ ]:
train_processed

In [ ]:
inputs = Input(shape=(6144, 1))
x = BatchNormalization()(inputs)

x1 = Conv1D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')(x)
x1 = MaxPooling1D(pool_size=2)(x1)

x2 = Conv1D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')(x1)
x2 = MaxPooling1D(pool_size=2)(x2)

residual1 = Conv1D(filters=64, kernel_size=5, strides=8, padding='same', activation='relu')(x)  # Adjust strides/kernel_size to match dimensions

x3 = Conv1D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')(x2)
x3 = MaxPooling1D(pool_size=2)(x3)
x3 = Add()([x3, residual1])

x4 = Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu')(x3)
x4 = MaxPooling1D(pool_size=2)(x4)

x5 = Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu')(x4)

residual2 = Conv1D(filters=512, kernel_size=5, strides=1, padding='same', activation='relu')(x4)  # Adjust strides/kernel_size to match dimensions

x6 = Conv1D(filters=512, kernel_size=5, strides=1, padding='same', activation='relu')(x5)
print(x6.shape)
print(residual2.shape)

x6 = Add()([x6, residual2])

x7 = Flatten()(x6)
x7 = Dropout(0.3)(x7)
x7 = Dense(512, activation='relu')(x7)

output = Dense(8, activation='linear')(x7)  
model = Model(inputs=inputs, outputs=output)

In [ ]:
tf.random.set_seed(42)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
drop_cols = major_oxides + ["ID", "Sample Name"]

X_train = train_processed.drop(columns=drop_cols)
y_train = train_processed[major_oxides]

X_test = test_processed.drop(columns=drop_cols)
y_test = test_processed[major_oxides]

In [ ]:
#print(X_train.shape)
X_train_reshaped = X_train.to_numpy().reshape(1538, 6144, 1)
X_test_reshaped = X_test.to_numpy().reshape(390, 6144, 1)

In [ ]:
import tensorflow as tf

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=6, restore_best_weights=True)

history = model.fit(X_train_reshaped, y_train,
                    epochs=1000,
                    batch_size=256,
                    validation_split=0.2, callbacks=[callback])

In [ ]:
predictions = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, predictions)

In [ ]:
for i, oxide in enumerate(major_oxides):
    y_test_oxide = y_test[oxide]
    predictions_oxide = predictions[:, i]
    rmse = mean_squared_error(y_test_oxide, predictions_oxide, squared=False)
    print(f"RMSE for {oxide}: {rmse}")